# Webscrapping for city data from Wikipedia

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Initialize a pandas DataFrame with specified columns
city_df = pd.DataFrame(columns=["city", "country", "population", "latitude", "longitude"])

# Function to scrape data from Wikipedia for a given city
def citygrab(city):
    # Construct the Wikipedia URL for the given city
    url = f"https://en.wikipedia.org/wiki/{city}"

    # Send a request to the URL and get the response
    response = requests.get(url)

    # Parse the response content using BeautifulSoup
    soup_city = BeautifulSoup(response.content, 'html.parser')

    # Initialize variables
    population = country = latitude = longitude = None

    # Extract population information
    for header in soup_city.find_all(class_="infobox-header"):
        if header.find(string=re.compile("population", re.IGNORECASE)):
            population = header.find_next(class_="infobox-data").text

    # Extract country information
    for merger in soup_city.find_all(class_="mergedtoprow") + soup_city.find_all(class_="mergedrow"):
        if merger.find(string=re.compile("country", re.IGNORECASE)):
            country = merger.find_next(class_="infobox-data").text

    # Extract latitude and longitude information
    for coords in soup_city.find_all(class_="infobox-full-data"):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            latitude = coords.find_next(class_="latitude").text
            longitude = coords.find_next(class_="longitude").text

    # Append the data as a new row to the city_df DataFrame
    city_df.loc[len(city_df)] = {
        "city": city.capitalize(),
        "country": country,
        "population": population,
        "latitude": latitude,
        "longitude": longitude
    }

# Define a list of city names you want to scrape data for
city_names = ["Berlin", "London"]

# Iterate through the list of city names and call the function for each city
for city in city_names:
    citygrab(city)

In [2]:
city_df

,city,country,population,latitude,longitude
0,Berlin,Germany,"3,755,251",52°31′12″N,13°24′18″E
1,London,England,"8,799,800[1]",51°30′26″N,0°7′39″W


In [3]:
#formating the data 
city_df['population'] = city_df['population'].str.replace('[1]','')
city_df['population'] = city_df['population'].str.replace(',','')
city_df["population"] = pd.to_numeric(city_df["population"])
city_df

,city,country,population,latitude,longitude
0,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,London,England,8799800,51°30′26″N,0°7′39″W


In [4]:
city_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        2 non-null      object
 1   country     2 non-null      object
 2   population  2 non-null      int64 
 3   latitude    2 non-null      object
 4   longitude   2 non-null      object
dtypes: int64(1), object(4)
memory usage: 96.0+ bytes


In [5]:
#creating a connection string to sql
import my_stuff

schema = "gans"
host = "127.0.0.1"
user = "root"
password = my_stuff.my_sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [7]:
#sending city data to city table in sql
city_df.to_sql(name='city',if_exists='append',con=connection_string,index=False) 

2

# Creating the Airport data

In [8]:
#read from sql
cities_df = pd.read_sql('city',con=connection_string)
cities_df

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,2,London,England,8799800,51°30′26″N,0°7′39″W


In [9]:
airports_dict = {'ICAO':[],'city_id':[]}

icao_dict = { 'Berlin':'EDDB',
             'London':'EGLL' }

In [10]:
for i, row in cities_df.iterrows():
    airports_dict['city_id'].append(row['city_id'])
    airports_dict['ICAO'].append(icao_dict[row['city']])

In [11]:
airports_dict

{'ICAO': ['EDDB', 'EGLL'], 'city_id': [1, 2]}

In [12]:
airports_df = pd.DataFrame(airports_dict)
airports_df

,ICAO,city_id
0,EDDB,1
1,EGLL,2


In [13]:
#send to sql
airports_df.to_sql('airport',if_exists='append',index=False, con=connection_string)

2